In [14]:
import numpy as np
import trimesh
import pyvista as pv
from skimage import measure
from skimage.draw import ellipsoid

In [15]:
def SurfaceOrientationTensor(vertices, faces):
    facesNo= faces.shape[0]
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    Area = mesh.area_faces
    face_normals = mesh.face_normals
    f = np.zeros((3,3))
    
    for k in range(0,facesNo):
        f = f + Area[k]*np.tensordot(face_normals[k,:],face_normals[k,:],axes=0) #outer product
    f = f/(np.sum(Area)) #normalise to the total surface area
    return f

def principalSOT(f):
    eigenVal, vectors = np.linalg.eig(f) #eigen analysis
    eigenValues = np.sort(eigenVal)[::-1] #sort eigenvalues in descending order

    f1 = eigenValues[0] #Largerst eigenvalue
    f2 = eigenValues[1] #Intermediate eigenvalue
    f3 = eigenValues[2] #Smallest eigenvalue

    C = f3/f1 #Compactness
    F = (f1-f3)/f1 #Flakiness
    E = (f2-f3)/f1 #Elongation
    return C, F, E

In [27]:
#Alternative definition

def surface_orientation_tensor(vertices, faces):
    face_normals = np.cross(vertices[faces[:, 1]] - vertices[faces[:, 0]], 
                            vertices[faces[:, 2]] - vertices[faces[:, 0]])
    face_normals /= np.linalg.norm(face_normals, axis=1).reshape(-1, 1)

    face_areas = 0.5 * np.linalg.norm(np.cross(vertices[faces[:, 1]] - vertices[faces[:, 0]], 
                                               vertices[faces[:, 2]] - vertices[faces[:, 0]]), axis=1)
    f = np.zeros((3, 3))
    for i in range(len(faces)):
        f += face_areas[i] * np.outer(face_normals[i], face_normals[i])
    f /= np.sum(face_areas)

    np.set_printoptions(precision=16)
    return f

def principalS_O_T(S):
    eigenvalues, _ = np.linalg.eig(S)
    eigenvals = np.sort(eigenvalues)[::-1]
    
    c = eigenvals[2] / eigenvals[0]
    f = (eigenvals[0] - eigenvals[2]) / eigenvals[0]
    e = (eigenvals[1] - eigenvals[2]) / eigenvals[0]
    return c, f, e


In [28]:
# generate ellipsoid
from skimage import measure
from skimage.draw import ellipsoid

ellip = ellipsoid(10, 20, 30, levelset=True)

# get vertices and faces
verts, faces, normals, values = measure.marching_cubes(ellip, 0.0, step_size=2)


In [69]:
from mayavi import mlab
x= [vert[0] for vert in verts]
y= [vert[1] for vert in verts]
z= [vert[2] for vert in verts]

mlab.triangular_mesh(x,y,z, faces, representation = 'wireframe')
mlab.show()

In [18]:
# calculate surface orientation tensor
S = SurfaceOrientationTensor(verts, faces)
# calculate shape descriptors
Compactness, Flakiness, Elongation = principalSOT(S)

# print results
print("Surface Tensor:", S)
print("Compactness:", Compactness)
print("Flakiness:", Flakiness)
print("Elongation:", Elongation)


Surface Tensor: [[ 6.61840606e-01 -3.35190776e-15 -4.52965734e-14]
 [-3.35190776e-15  2.24965088e-01 -1.43340866e-14]
 [-4.52965734e-14 -1.43340866e-14  1.13194306e-01]]
Compactness: 0.17102955706849332
Flakiness: 0.8289704429315067
Elongation: 0.16887870155518597


In [30]:
S_alt = surface_orientation_tensor(verts,faces)
# calculate shape descriptors
Compactness_ALT, Flakiness_ALT, Elongation_ALT = principalS_O_T(S_alt)

print("Surface Tensor alt:", S_alt)
print("Compactness_ALT:", Compactness_ALT)
print("Flakiness_ALT:", Flakiness_ALT)
print("Elongation_ALT:", Elongation_ALT)

Surface Tensor alt: [[ 6.6184060987049098e-01 -1.1339820683263090e-11  8.1329649451130286e-11]
 [-1.1339820683263090e-11  2.2496508960338529e-01  7.7067851753142563e-11]
 [ 8.1329649451130286e-11  7.7067851753142563e-11  1.1319430691866403e-01]]
Compactness_ALT: 0.17102955791850546
Flakiness_ALT: 0.8289704420814946
Elongation_ALT: 0.16887870133353192
